<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/main/Reddit_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment

In [1]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 911kB 5.8MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/g

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [2]:
url = 'https://www.reddit.com/r/wallstreetbets/search/?q=flair%3A%22Daily%20Discussion%22&restrict_sr=1&sort=new'
driver.get(url)

In [3]:
from datetime import date, timedelta
from dateutil.parser import parse

In [14]:
yesterday = date.today() - timedelta(days=1)
links = driver.find_elements_by_xpath('//*[@class="_eYtD2XCVieq6emjKBH3m"]')

In [28]:
for a in links:
  if a.text.startswith('Daily Discussion Thread'):
    date = "".join(a.text.split(' ')[-3:])
    parsed = parse(date) 
    if parse(str(yesterday)) == parsed:
      link = a.find_element_by_xpath('../..').get_attribute('href')
    
  if a.text.startswith('Weekend'):
    weekend_date = a.text.split(' ')
    parsed_date = weekend_date[-3] + ' ' + \
      weekend_date[-2].split("-")[1] + weekend_date[-1]
    parsed = parse(parsed_date) 
    saturday = weekend_date[-3] + ' ' + \
      str(int(weekend_date[-2].split("-")[1]
      .replace(',','')) - 1) + ' ' + weekend_date[-1] 

    if parse(str(yesterday)) == parsed: 
      link = a.find_element_by_xpath('../..').get_attribute('href')
    elif parse(str(yesterday)) == parse(str(saturday)):
      link = a.find_element_by_xpath('../..').get_attribute('href')